In [1]:
#libraries
#!pip install transformers==4.31.0 
#!pip install sentence-transformers==2.2.2 

In [2]:
#specific version to avoid error with vector store
#!pip install chromadb==0.5.3
#!pip install datasets

In [3]:
#loading training data for providing context to the model
import pandas as pd
train_data_df=pd.read_csv("C:/Users/USER/Documents/Projects/Instruction Dataset/LLM Training Data/train_data_split.csv")
## Convert the DataFrame to a format compatible with Hugging Face Datasets
train_data_df['Instruction'] = train_data_df['Instruction']+ ". Input Text: " + train_data_df['input_text'] + "\n\nExtracted Properties:\n\n"+ train_data_df['Output'].astype(str)
train_dataset_dict = {
    'instruction': train_data_df['Instruction'].tolist()
}

In [4]:
#verify
#train_dataset_dict

In [5]:
#creating context document from the list
docs=train_dataset_dict['instruction']

In [6]:
#pip install typing_extensions==4.7.1 --upgrade
#!pip install -U pandas-profiling
#!pip install typing-extensions --upgrade

In [7]:
#initializing the embedding model
from sentence_transformers import SentenceTransformer
#pip install chromadb==0.5.3
import chromadb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder = '/data/base_models')

C:\Users\USER\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
#do the embedding
embeddings = embedding_model.encode(docs)

In [9]:
#verify the embeddings
embeddings.shape

(1040, 768)

In [10]:
#indexing-run this only once
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="q-rag_GPT")

In [11]:
#addding the train data
collection.add(
    embeddings = embeddings,
    documents=docs,
    ids= [str(i) for i in range(len(docs))]
)

In [27]:
#retrieving
def retrieve_vector_db(query, n_results=3):
    results = collection.query(
    query_embeddings = embedding_model.encode(query).tolist(),
    n_results=n_results
    )
    return results['documents']

In [36]:
#sample query
#we can have different queries depending on the property we are interested in
#query=input ("Please Enter your query here: ") 
query = "Please extract the working temperature of the quantum cascade laser device. Please print none if the value is not in text and do not give any explanations. In the output, just include only the extracted property."
#input_text is the user input text containing the properties for extraction
#print("Enter the text containing the properties herer")
input_text= "2.1 THz quantum cascade laser (QCL) based on a scatterin gassisted injection and resonant-phonon depopulation design scheme is demonstrated. The QCL is based on a four-well period implemented in the GaAs/Al0:15Ga0:85As material system. The QCL operates up to a heat-sink temperature of 144 K in pulsed-mode, which is considerably higher than that achieved for previously reported THz QCLs operating around the frequency of 2 THz. At 46 K, the threshold current-density was measured as \~ 745 A/cm2 with a peak-power output of \~ 10 mW. Electrically stable operation in a positive differential-resistance regime is achieved by a careful choice of design parameters. The results validate the robustness of scattering-assisted injection schemes for development of low-frequency (n < 2:5 THz) QCLs."
query=query + input_text
retrieved_results = retrieve_vector_db(query)
print(query)

Please extract the working temperature of the quantum cascade laser device. Please print none if the value is not in text and do not give any explanations. In the output, just include only the extracted property.2.1 THz quantum cascade laser (QCL) based on a scatterin gassisted injection and resonant-phonon depopulation design scheme is demonstrated. The QCL is based on a four-well period implemented in the GaAs/Al0:15Ga0:85As material system. The QCL operates up to a heat-sink temperature of 144 K in pulsed-mode, which is considerably higher than that achieved for previously reported THz QCLs operating around the frequency of 2 THz. At 46 K, the threshold current-density was measured as \~ 745 A/cm2 with a peak-power output of \~ 10 mW. Electrically stable operation in a positive differential-resistance regime is achieved by a careful choice of design parameters. The results validate the robustness of scattering-assisted injection schemes for development of low-frequency (n < 2:5 THz)

In [29]:
#viewing the retreived 3 sample context text based on the user query.
retrieved_results[0]

['From the following sentence, please extract the lasing frequency of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text. Input Text: By utilizing a bound-to-continuum active region and semi-insulating surface-plasmon waveguide design, we have successfully demonstrated the prolonged functioning of terahertz quantum cascade lasers emitting at 3.2 THz in continuous-wave mode.\t\n\nExtracted Properties:\n\n3.2 THz',
 'From the following sentence, please extract the lasing frequency of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text. Input Text: Using a bound-to-continuum active region and high-resistivity surface-plasmon waveguide design, we were able to achieve continuous-wave operation of 10.5 THz quantum cascade lasers emitting at 5.0 THz.\n\nExtracted Properties:\n\n10.5 THz, 5.0 THz',
 'From the following sentence, please extract the lasing frequency of the Quantum Cascade S

In [30]:
#assigning the context
context=retrieved_results[0]

In [31]:
#prompt template
prompt = f'''
[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations. The output format should correspond to the one in the example sentences. Example sentences containing an instruction, input text and the extracted properties are given below:

Example Sentences: {context}

Instruction: {query}

[/INST]
'''

In [32]:
#view prompt
print(prompt)


[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations. The output format should correspond to the one in the example sentences. Example sentences containing an instruction, input text and the extracted properties are given below:

Example Sentences: ['From the following sentence, please extract the lasing frequency of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text. Input Text: By utilizing a bound-to-continuum active region and semi-insulating surface-plasmon waveguide design, we have successfully demonstrated the prolonged functioning of terahertz quantum cascade lasers emitting at 3.2 THz in continuous-wave mode.\t\n\nExtracted Properties:\n\n3.2 THz', 'From the following sentence, please extract the lasing frequency of the Quantum Cascade Semiconduct

In [33]:
#parsing the complete prompt to GPT
#GPT
import openai
import os
from IPython.display import Markdown
api_key="OPEN_API_KEY"

In [34]:
#some function to chat with GPT
import openai

# Set your API key securely
openai.api_key = api_key

def chatWithGPT4(user_text, print_output=False):
    try:
        # Creating text completions using the updated `ChatCompletion` class
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": user_text}],
            max_tokens=3000
        )
        text_output = response['choices'][0]['message']['content'] if response['choices'] else "No response generated."
        if print_output:
            print(text_output)
        return text_output
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [35]:
# Extracting a property
result = chatWithGPT4(prompt)
print(result)

Extracted Properties:

longitudinal-optical (LO) phonon scattering
